In [1]:
# Importando bibliotecas necessárias para a construção de grafos
import netpixi
from netpixi.integration.gt import *
import pandas as pd
import numpy as np
import csv
import ast
import cpnet
from graph_tool import spectral

<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
/usr/local/lib/python3.10/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
/usr/local/lib/python3.10/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
<frozen importlib._bootstrap>:914: ImportWarning: VendorImporter.find_spec() not found; falling back to find_module()
/usr/local/lib/python3.10/site-packages/seaborn/cm.py:1582: PendingDeprecationWarning: The register_cmap function will be deprecated in a future version. Use ``matplotlib.colormaps.register(name)`` instead.
  mpl_cm.register_cmap(_name, _cmap)
/usr/local/lib/python3.10/site-packages/seaborn/cm.py:1583: PendingDeprecationWarning: The register_cmap function will b

In [44]:
df.loc[~df['Critic_Score'].isna()] 

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16700,Breach,PC,2011.0,Shooter,Destineer,0.01,0.00,0.00,0.00,0.01,61.0,12.0,5.8,43.0,Atomic Games,T
16701,Bust-A-Move 3000,GC,2003.0,Puzzle,Ubisoft,0.01,0.00,0.00,0.00,0.01,53.0,4.0,NaN,NaN,Taito Corporation,E
16702,Mega Brain Boost,DS,2008.0,Puzzle,Majesco Entertainment,0.01,0.00,0.00,0.00,0.01,48.0,10.0,NaN,NaN,Interchannel-Holon,E
16706,STORM: Frontline Nation,PC,2011.0,Strategy,Unknown,0.00,0.01,0.00,0.00,0.01,60.0,12.0,7.2,13.0,SimBin,E10+


In [45]:
df = pd.read_csv('./Video_Games_Sales_as_at_22_Dec_2016.csv')
df.dropna(subset=['Publisher'], inplace=True)
df = df.loc[~df['Critic_Score'].isna() & ~df['User_Score'].isna()]

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7013 entries, 0 to 16709
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             7013 non-null   object 
 1   Platform         7013 non-null   object 
 2   Year_of_Release  6893 non-null   float64
 3   Genre            7013 non-null   object 
 4   Publisher        7013 non-null   object 
 5   NA_Sales         7013 non-null   float64
 6   EU_Sales         7013 non-null   float64
 7   JP_Sales         7013 non-null   float64
 8   Other_Sales      7013 non-null   float64
 9   Global_Sales     7013 non-null   float64
 10  Critic_Score     7013 non-null   float64
 11  Critic_Count     7013 non-null   float64
 12  User_Score       7013 non-null   float64
 13  User_Count       7013 non-null   float64
 14  Developer        7009 non-null   object 
 15  Rating           6943 non-null   object 
dtypes: float64(10), object(6)
memory usage: 931.4+ KB


In [47]:
df.groupby('Genre')['Name'].count().sort_values(ascending=False)

Genre
Action          1677
Sports           973
Shooter          886
Role-Playing     721
Racing           598
Platform         406
Misc             396
Fighting         382
Simulation       306
Strategy         282
Adventure        265
Puzzle           121
Name: Name, dtype: int64

In [48]:
df_grouped_Genre = df.groupby(['Genre']).agg({'Global_Sales': np.mean})
df_grouped_Genre.sort_values('Global_Sales', ascending=False)

,Global_Sales
Genre,
Misc,1.074444
Platform,0.931429
Shooter,0.930305
Sports,0.876084
Racing,0.806756
Action,0.731431
Role-Playing,0.699251
Simulation,0.670163
Fighting,0.656571


In [49]:
sports_mean_sales = df_grouped_Genre.loc['Sports']['Global_Sales']
action_mean_sales = df_grouped_Genre.loc['Action']['Global_Sales'] 
rpg_mean_sales = df_grouped_Genre.loc['Role-Playing']['Global_Sales'] 

In [50]:
def segregate_genre_to_list(df, genre, df_list, threshold=None):
    seg_df = df[df['Genre'] == genre]
    if threshold:
        seg_df = seg_df[seg_df['Global_Sales'] > threshold]
    df_list.append(seg_df)

In [51]:
sorted_years = df['Year_of_Release'].unique()
sorted_years.sort()
print(sorted_years)

[1985. 1988. 1992. 1994. 1996. 1997. 1998. 1999. 2000. 2001. 2002. 2003.
 2004. 2005. 2006. 2007. 2008. 2009. 2010. 2011. 2012. 2013. 2014. 2015.
 2016.   nan]


In [52]:
df_list = []
sport_df_list = []
action_df_list = []
rpg_df_list = []
margem = 4
for i in range(0, len(sorted_years), margem):
    print(i)
    df_list.append(df[df['Year_of_Release'].isin(sorted_years[i:i+margem])])


0
4
8
12
16
20
24


In [53]:
df_list[0]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
1567,Battle Arena Toshinden,PS,1994.0,Fighting,Sony Computer Entertainment,0.39,0.26,0.53,0.08,1.27,69.0,4.0,6.3,4.0,Tamsoft,T
14472,Alter Ego,PC,1985.0,Simulation,Activision,0.00,0.03,0.00,0.01,0.03,59.0,9.0,5.8,19.0,"Viva Media, Viva Media, LLC",T
14612,Doom,PC,1992.0,Shooter,id Software,0.02,0.00,0.00,0.00,0.03,85.0,44.0,8.2,1796.0,id Software,M
14623,SimCity,PC,1988.0,Simulation,Maxis,0.00,0.02,0.00,0.01,0.03,64.0,75.0,2.2,4572.0,Maxis,E10+


In [54]:
publisher_platforms = []
for db in df_list:
    publisher_platforms.append(db.groupby('Platform')['Publisher'].unique().to_dict())
publisher_platforms

[{'PC': array(['Activision', 'id Software', 'Maxis'], dtype=object),
  'PS': array(['Sony Computer Entertainment'], dtype=object)},
 {'DC': array(['Sega', 'Namco Bandai Games'], dtype=object),
  'PC': array(['Vivendi Games', 'Activision', 'Eidos Interactive',
         'Microsoft Game Studios', 'Interplay', 'Empire Interactive'],
        dtype=object),
  'PS': array(['Sony Computer Entertainment', 'SquareSoft', 'GT Interactive',
         'Konami Digital Entertainment', 'Virgin Interactive', 'Activision',
         'Eidos Interactive', 'Take-Two Interactive', 'Electronic Arts',
         'Namco Bandai Games', 'Hasbro Interactive', 'Tecmo Koei',
         'Enix Corporation', 'Square EA', 'Kadokawa Shoten',
         'ASCII Entertainment', 'Psygnosis', 'Ubisoft',
         'Human Entertainment', 'Midas Interactive Entertainment',
         'Acclaim Entertainment', 'Titus', 'ESP'], dtype=object)},
 {'DC': array(['Sega', 'Virgin Interactive', 'Ubisoft'], dtype=object),
  'GBA': array(['Nintendo', 

In [55]:
publishers = df["Publisher"].unique()
publishers

array(['Nintendo', 'Microsoft Game Studios', 'Take-Two Interactive',
       'Sony Computer Entertainment', 'Activision', 'Ubisoft',
       'Bethesda Softworks', 'Electronic Arts', 'SquareSoft',
       'GT Interactive', 'Konami Digital Entertainment', 'Square Enix',
       'Sony Computer Entertainment Europe', 'Virgin Interactive',
       'LucasArts', '505 Games', 'Capcom',
       'Warner Bros. Interactive Entertainment', 'Universal Interactive',
       'RedOctane', 'Atari', 'Eidos Interactive', 'Namco Bandai Games',
       'Vivendi Games', 'MTV Games', 'Sega', 'THQ',
       'Disney Interactive Studios', 'Acclaim Entertainment',
       'Midway Games', 'Deep Silver', 'NCSoft', 'Tecmo Koei',
       'Valve Software', 'Infogrames', 'Mindscape', 'Valve',
       'Hello Games', 'Global Star', 'Gotham Games',
       'Crave Entertainment', 'Hasbro Interactive', 'Codemasters',
       'TDK Mediactive', 'Zoo Games', 'Sony Oznline Entertainment', 'RTL',
       'D3Publisher', 'Unknown', 'Black Label 

In [56]:
df_list[1]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
52,Gran Turismo,PS,1997.0,Racing,Sony Computer Entertainment,4.02,3.87,2.54,0.52,10.95,96.0,16.0,8.7,138.0,Polyphony Digital,E
65,Final Fantasy VII,PS,1997.0,Role-Playing,Sony Computer Entertainment,3.01,2.47,3.28,0.96,9.72,92.0,20.0,9.2,1282.0,SquareSoft,T
67,Gran Turismo 2,PS,1999.0,Racing,Sony Computer Entertainment,3.88,3.42,1.69,0.50,9.49,93.0,23.0,9.0,135.0,Polyphony Digital,T
88,Final Fantasy VIII,PS,1999.0,Role-Playing,SquareSoft,2.28,1.72,3.63,0.23,7.86,90.0,24.0,8.6,644.0,SquareSoft,T
106,Tekken 3,PS,1998.0,Fighting,Sony Computer Entertainment,3.27,2.22,1.40,0.29,7.16,96.0,15.0,9.1,367.0,Namco,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13310,Saiyuki: Journey West,PS,1999.0,Strategy,Tecmo Koei,0.03,0.02,0.00,0.00,0.05,73.0,9.0,7.1,9.0,Koei,E
13337,StarCraft,PC,1998.0,Strategy,Activision,0.00,0.04,0.00,0.01,0.05,88.0,15.0,9.1,813.0,Blizzard Entertainment,T
14214,Planescape: Torment,PC,1998.0,Role-Playing,Interplay,0.01,0.02,0.00,0.00,0.03,91.0,22.0,9.2,813.0,Black Isle Studios,T
15377,Homeworld,PC,1998.0,Strategy,Vivendi Games,0.00,0.02,0.00,0.00,0.02,93.0,20.0,9.0,326.0,Relic,E


In [57]:
df['Name'].loc[df["Publisher"] == 'Nintendo'].nunique() 

292

In [58]:
action_graph = Graph(directed=False) # não-dirigido
action_graph.add_vp('Publisher')
action_graph.add_vp('Games_released')
action_graph.add_ep('strength')

for pub in publishers:
    action_graph.add_vertex(pub)
    action_graph.get_vertex(pub)['Publisher'] = pub
    num_games = df['Name'].loc[df["Publisher"] == pub].nunique() 
    action_graph.get_vertex(pub)['Games_released'] = num_games

# for df in df_list:
#     for pub in publishers:
#         for pub2 in publishers:
#             if pub != pub2 and not action_graph.has_edge(pub, pub2):
#                 action_graph.add_edge(pub, pub2)

In [59]:
for platform_dict in publisher_platforms:
    for platform, publishers in platform_dict.items():
        for i in range(len(publishers)):
            if i != len(publishers)-1:
                next_vertex = i+1
                while(next_vertex < len(publishers)):
                    if(publishers[i] != publishers[next_vertex] and not action_graph.has_edge(publishers[i], publishers[next_vertex])):
                        action_graph.add_edge(publishers[i], publishers[next_vertex])
                        action_graph.get_edge(publishers[i], publishers[next_vertex])['strength'] = 1
                        next_vertex += 1
                    else:
                        action_graph.get_edge(publishers[i], publishers[next_vertex])['strength'] += 1
                        next_vertex += 1

In [60]:
print('número de vértices de g:', action_graph.num_vertices())

número de vértices de g: 273


In [61]:
print('número de arestaas de g:', action_graph.num_edges())

número de arestaas de g: 11124


In [62]:
gt_save(action_graph, 'action_graph.net.gz')

In [63]:
g = gt_load('action_graph.net.gz')
g = gt_clean(g)
m = gt_draw.sfdp_layout(g)
gt_move(g, m)
gt_save(g, 'action_sfdp.net.gz')

In [64]:
r = netpixi.render('action_sfdp.net.gz', infinite=True);

In [65]:
r.vertex_scale('Games_released', 20, 50)
r.edge_scale('strength', 0.3, 4)
r.vertex_default(color=0xff8674, kscale = 4, bwidth=3, bcolor=0x000000)
r.edge_default(curve1=1, curve2=1)
r.vertex_set_key('Publisher')